# Functions for Diagnostic Indicators

### Functions regarding import data

In [5]:
def prepare_data(database, startyear=2010, dt=10, Region='European', **kwargs):
    data_raw = pd.read_csv(database, **kwargs)
    
    # Choose only decadal data
    data = data_raw.loc[:,['Model', 'Scenario', 'Region', 'Variable', 'Unit']+[str(y) for y in np.arange(startyear, 2101, dt)]]
    
    # Choose only region == World
    if Region == 'European':
        data = data[data['Region'].isin(['Europe', 'Europe (excl. Turkey)', 'EU27', 'Europe (incl. Turkey)', 'all'])]
    
    # Set all scenario names to lowercase
    data['Scenario'] = data['Scenario'].str.lower()
    
    # Add column Name, equal to Model + Scenario
    data.insert(2, 'Name', data['Model'] + ' ' + data['Scenario'])
    
    return data

def interpolate_missing_5years(data):
    col_2010 = list(data.columns).index('2010')
    for i, year in enumerate(range(2015, 2105, 10)):
        data.insert(col_2010+2*i+1, str(year), data[[str(year-5), str(year+5)]].mean(axis=1))

In [6]:
def get(data, scenario, variable, year=None):
    selection = data[(data['Scenario'] == scenario) & (data['Variable'] == variable)].set_index('Model')
    if year is None:
        return selection.loc[:,'2010':]
    else:
        return selection[year]

In [7]:
def set_value_from_var_column(data, meta, metacol, col, year, scenario):
    """
    Adds a column to the meta df using the `data` df. Which variable to use
    is taken from a `metacol` column in the meta df. For example, the GDP
    metric is sometimes GDP|PPP and sometimes GDP|MER, depending on the model.
    
    In this example, metacol would be 'GDP_metric'.
    """
    meta[col] = np.nan
    for model, info in meta.iterrows():
        var = info[metacol]
        selection = data[
            (data['Variable'] == var)
            & (data['Model'] == model)
            & (data['Scenario'] == scenario)
        ]
        if len(selection) > 0:
            meta.loc[model, col] = selection.iloc[0][year]

In [8]:
def add_legend_item(fig, name='', mode='markers', **kwargs):
    """
    In Plotly, a legend item can be added manually by adding an empty trace
    """
    fig.add_scatter(x=[None], y=[None], name=name, mode=mode, **kwargs)

## Function for the indicator comparison

In [10]:
def add_model_comparison(
    fig, fig_col, meta_col,
    label_posx=None, label_width=None,
    narrative_left=None, narrative_right=None,
    shared_yaxes=False, showlegendlines=True,
    labelshift=1, exclude_models=None, models=models
):
    """
    Used to create the right part of each figure: the indicator comparison.
    This is shared code for each indicator.
                        
    fig:            Plotly subplot figure
    fig_col:        typically will be 2 if the right subplot is used
    meta_col:       column from meta df used to get the indicator values
    label_posx [None]: by default, the position (relative to the x-axis of col fig_col)
            is calculated automatically based on the spread of indicator values. This can
            be overrided manually if this calculation doesn't work properly.
    narrative_left: string for arrow pointing left of median value of indicators
    narrative_right: same for right
    shared_yaxes:   if the left subplot has to share the same y-axis, set to True
    showlegendlines: while the legend is always shown, the coloured lines are not always
            necessary. Setting this to False hides these lines.
    labelshift:     shift the labels a bit more ( > 1) or less ( < 1) to the left.
    exclude_models: list of model names that should be excluded from this plot
    models:         by default the normal `models` dataframe, but can be used as override.
    """
    
    if exclude_models is not None:
        models = models[~models['Full model'].str.contains(exclude_models)].copy()
        models['i'] = np.arange(len(models))
    
    n = models['i'].max()
    meta_selection = meta[meta['Stripped model'].isin(models.index)]

    # Add legend items
    for name, symbol, size in [('Newest', 'star', 8), ('Older version', 'circle', 4)]:
        add_legend_item(fig, name, marker={'symbol': symbol, 'size': size, 'color': 'black'}, legendgroup='Age')

    # Add shade for 1-sigma range
    q0, median, q1 = meta_selection[meta_col].quantile([0.16, 0.5, 0.84])
    # mean = meta_selection[meta_col].mean()
    fig.add_scatter(
        x=[q0, q0, q1, q1], y=[-1, n+1, n+1, -1],
        fill='toself', fillcolor='rgba(0,0,0,.1)',
        line_width=0, mode='lines',
        name='16-84th perc.', row=1, col=fig_col
    )

    # Add lines for median and mean
    fig.add_scatter(
        x=[median, median], y=[-1, n+1],
        mode='lines', line={'color': '#888', 'width': 2},
        name='Median', row=1, col=fig_col
    )
    
    # Calculate position of legend items
    vmin, vmax = meta_selection[meta_col].min(), meta_selection[meta_col].max()
    label_posx = vmin - 0.66 * labelshift * (vmax - vmin) if label_posx is None else label_posx
    label_width = 0.15 * (vmax - vmin) if label_width is None else label_width

    for model, (modeltype, fullmodel, i, color) in models.iterrows():

        selection = meta_selection[meta_selection['Stripped model'] == model]

        # Add dots and stars
        fig.add_scatter(
            x=selection[meta_col], y=[i] * len(selection),
            marker={
                'color': color, 'opacity': 1,
                'symbol': ['star' if is_newest else 'circle' for is_newest in selection['Newest']],
                'size': [12 if is_newest else 7 for is_newest in selection['Newest']],
                'line': {'color': '#FFF', 'width': 1}
            },
            mode='markers', showlegend=False,
            row=1, col=fig_col
        )

        # Add legend line
        if showlegendlines:
            fig.add_scatter(
                x=[label_posx-label_width, label_posx], y=[i, i],
                mode='lines', line={'color': color, 'width': 3},
                row=1, col=fig_col, showlegend=False
            )

        # Name of model
        fig.add_annotation(
            text=model, 
            x=label_posx, y=i, xanchor='left',
            row=1, col=fig_col,
            bgcolor='#FFF',
            showarrow=False
        )

    

    # Add model type brackets
    x_max = meta_selection[meta_col].max()
    dx = x_max - meta_selection[meta_col].min()
    
    x_right = 0.05 * dx + x_max # 6% to the right of the most right point
    x_width = 0.03 * dx
    for modeltype, selection in models.groupby('Type'):
        first, last = selection['i'].min(), selection['i'].max()
        # Bracket itself
        dy = 0.3
        fig.add_scatter(
            x=[x_right, x_right+x_width, x_right+x_width, x_right], y=[first-dy, first-dy, last+dy, last+dy],
            mode='lines', line_color='#999', showlegend=False,
            row=1, col=fig_col
        )
        # Name of model type
        fig.add_annotation(
            x=x_right+1.25*x_width, y=(first+last)/2,
            text=modeltype, textangle=90,
            bgcolor='#FFF',
            showarrow=False, yanchor='middle', xanchor='left',
            row=1, col=fig_col
        )
        
        
    
    # Add narrative arrows
    for label, toLeft in [(narrative_left, True), (narrative_right, False)]:
        if label is None:
            continue
        arrowlength=65
        multiplier = -1 if toLeft else 1
        fig.add_annotation(
            xref=f'x{fig_col}', yref='paper', xanchor='center', yanchor='top',
            x=median, y=-0.08, ax=arrowlength*multiplier, ay=0, xshift=multiplier*5,
            width=arrowlength*2, align='right' if toLeft else 'left',
            text=label,
            showarrow=True, arrowside='start'
        )
        
    # Update layout
    fig.update_yaxes(
        col=None if shared_yaxes else fig_col,
        gridcolor=gridcolor,
        tickvals=models['i'],
        range=[n+1, -1],
        zeroline=False,
        showticklabels=False
    ).update_layout(
        legend={'tracegroupgap': 0, 'y': 0.5},
    )



NameError: name 'models' is not defined

## Indicator: relative abatement index

$$RAI(t) = \frac{CO_2\text{FFI Base}(t)-CO_2\text{FFI Pol}(t)}{CO_2\text{FFI Base}(t)}$$

In [ ]:
def calc_relative_abatement_index(data, year, pol='diag-c80-gr5', base='diag-base', var=var_CO2_FFI):
    
    # Get CO2 FFI Base and Pol
    CO2_FFI_base = get(data, base, var, year)
    CO2_FFI_pol  = get(data, pol, var, year)
    
    return (CO2_FFI_base - CO2_FFI_pol) / CO2_FFI_base

In [ ]:
def create_fig_RAI(
    year: str, var, var_name='CO2 FFI', 
    narrative_left='Less CO<sub>2</sub> reduction', narrative_right='More CO<sub>2</sub> reduction',
    xrange=None, **kwargs
):
    
    
    # Calculate indicators
    col_c30_RAI = f'RAI c30 {year} {var_name}'
    col_c80_RAI = f'RAI c80 {year} {var_name}'
    col_c30_cprice = f'Carbon price c30 {year}'
    col_c80_cprice = f'Carbon price c80 {year}'
    
    meta[col_c30_RAI] = calc_relative_abatement_index(data, year, pol='diag-c30-gr5', var=var)
    meta[col_c80_RAI] = calc_relative_abatement_index(data, year, pol='diag-c80-gr5', var=var)
    meta[col_c30_cprice] = get(data, 'diag-c30-gr5', 'Price|Carbon', year)
    meta[col_c80_cprice] = get(data, 'diag-c80-gr5', 'Price|Carbon', year)
    meta.loc[meta[col_c30_cprice] == 0, col_c30_cprice] = np.nan
    meta.loc[meta[col_c80_cprice] == 0, col_c80_cprice] = np.nan
    
    # Create figure
    fig_RAI = make_subplots(1, 2, horizontal_spacing=0.02, column_widths=[0.4, 0.6], subplot_titles=(
        '<b>a.</b> Carbon price vs RAI<br> ', '<b>b.</b> RAI per model<br> '
    ))
    col_RAI_vs_cprice = 1
    col_RAI = 2


    ##############
    # 1a: RAI vs cprice for c30 and c80
    ##############


    curr_cols = [col_c30_RAI, col_c80_RAI, col_c30_cprice, col_c80_cprice]

    for is_newest, selection in meta[~meta[curr_cols].isna().any(axis=1)].groupby('Newest'):
        for i, (model, info) in enumerate(selection.iterrows()):
            stripped_model = info['Stripped model']

            if stripped_model not in models.index:
                # Ignore entries that are not in `models`
                continue

            color = models.loc[stripped_model, 'Color'] if is_newest else '#DDD'
            label = info['Stripped model'] if is_newest else 'Older model version'
            dash = 'solid'

            fig_RAI.add_scatter(
                x=[0, info[col_c30_RAI], info[col_c80_RAI]],
                # y=[0, info[col_c30_cprice], info[col_c80_cprice]],
                # Hardcoded carbon price:
                y=[0, 48.86, 130.31],
                line={'color': color, 'dash': dash},
                mode='lines',
                name=label, legendgroup=label, showlegend=False,
                row=1, col=col_RAI_vs_cprice
            )




    ##############
    # 1b: RAI based on c80
    ##############

    add_model_comparison(
        fig_RAI, col_RAI, col_c80_RAI,
        narrative_left=narrative_left, narrative_right=narrative_right,
        **kwargs
    )


    # Update layout

    (
        fig_RAI
        .update_xaxes(
            title=f'Relative Abatement Index in {year}',
            gridcolor=gridcolor,
            tickvals=np.arange(0, 2, 0.2),
            zeroline=True,
            title_standoff=40
        )
        .update_xaxes(
            col=col_RAI,
            range=xrange
        )
        .update_yaxes(
            col=col_RAI_vs_cprice,
            title=f'Carbon price in {year} (2010$/tCO<sub>2</sub>)'
        )
        .update_layout(
            width=860,
            height=400,
            margin={'l': 60, 'r': 30, 't': 50, 'b': 70},
            hovermode='closest'
        )
    )
    return fig_RAI

## Carbon intensity over energy intensity

$$\text{CoEI}=\frac{\text{Res}(CI)}{\text{Res}(EI)}$$

In [ ]:
def calc_carbon_and_energy_intensity(data, year, scenario):
    
    # For each model, get either GDP|PPP or GDP|MER (depending on column `GDP_metric`)
    GDP_column = f'GDP {year} {scenario}'
    set_value_from_var_column(data, meta, 'GDP_metric', GDP_column, year, scenario)
    
    CO2_FFI      = get(data, scenario, var_CO2_FFI, year)
    final_energy = get(data, scenario, 'Final Energy', year)
    GDP_PPP      = meta[GDP_column]
    
    carbon_intensity = CO2_FFI / final_energy
    energy_intensity = final_energy / GDP_PPP
    
    return carbon_intensity, energy_intensity

def calc_normalised_carbon_and_energy_intensity(data, year):
    CI_pol, EI_pol           = calc_carbon_and_energy_intensity(data, year, 'diag-c80-gr5')
    CI_baseline, EI_baseline = calc_carbon_and_energy_intensity(data, year, 'diag-base')
    return CI_pol / CI_baseline, EI_pol / EI_baseline

In [ ]:
##################
## Functions required to generate confidence ellipse
##################

def ellipse(a, b, npoints):
    x = np.linspace(-a, a, npoints)
    y1 = b * np.sqrt(1-(x/a)**2)
    y2 = -y1
    return np.concatenate([x,x[::-1]]), np.concatenate([y1,y2[::-1]])

def rotate(x, y, theta):
    return x*np.cos(theta)-y*np.sin(theta), x*np.sin(theta)+y*np.cos(theta)


def confidence_ellipse(x_values, y_values, nsigma, npoints=300):
    # Calculate center of confidence ellipse
    mu_x, mu_y = np.mean(x_values), np.mean(y_values)
    
    # Calculate correlation coefficient and covariances
    cov_matrix = np.cov([x_values, y_values])
    cov_xy = cov_matrix[0,1]
    sigma_x, sigma_y = np.sqrt(cov_matrix[0,0]), np.sqrt(cov_matrix[1,1])
    rho = cov_xy / (sigma_x * sigma_y)
    
    # Get the x-y points for the default ellipse with a=sqrt(1+rho), b=sqrt(1-rho)
    ellipse_x, ellipse_y = ellipse(np.sqrt(1+rho), np.sqrt(1-rho), npoints)
    
    # Rotate ellipse 45 degrees counter-clockwise
    ellipse_x, ellipse_y = rotate(ellipse_x, ellipse_y, np.pi/4)
    
    # Scale ellipse horizontally by (2*n*sigma_x) and vertically by (2*n*sigma_y)
    # Note: scaling by 2*n*sigma_x means that the x_values (centered around 0) should
    # be multiplied by n*sigma_x, not 2*n*sigma_x
    ellipse_x = nsigma*sigma_x * ellipse_x
    ellipse_y = nsigma*sigma_y * ellipse_y
    
    # Shift ellipse such that its center is situated at the point mu_x, mu_y
    ellipse_x += mu_x
    ellipse_y += mu_y
    
    return ellipse_x, ellipse_y

In [ ]:
def create_fig_CoEI(year: str, xrange=None, **kwargs):

    fig_CoEI = make_subplots(1, 2, horizontal_spacing=0.02, column_widths=[0.4, 0.6], subplot_titles=(
        '<b>a.</b> Energy Intensity vs Carbon Intensity<br> ', f'<b>b.</b> ERT per model ({year})<br> '
    ))

    ##############
    # 2a: CI vs EI scatter for C80-gr5 (2050 and 2100)
    ############## 

    # Add background shape for 2050 and 2100
    for col_CI, col_EI, col_year in [(col_CI_2050, col_EI_2050, 2050), (col_CI_2100, col_EI_2100, 2100)]:
        selection = meta[~meta[[col_CI, col_EI]].isna().any(axis=1)]
        x_ellipse, y_ellipse = confidence_ellipse(selection[col_CI], selection[col_EI], 2)
        fig_CoEI.add_scatter(
            x=x_ellipse, y=y_ellipse, fill='toself',
            showlegend=False, fillcolor='rgba(0,0,0,.08)', line_width=0
        )
        fig_CoEI.add_annotation(
            text=col_year, x=max(x_ellipse), y=min(y_ellipse),
            font_color='#999', showarrow=False
        )

    # Create connected dots for 2050 and 2100 values
    for is_newest, selection in meta.groupby('Newest'):
        for model, info in selection.iterrows():
            stripped_model = info['Stripped model']

            if stripped_model not in models.index:
                # Ignore entries that are not in `models`
                continue

            color = models.loc[stripped_model, 'Color'] if is_newest else '#BBB'

            fig_CoEI.add_scatter(
                x=info[[col_CI_2050, col_CI_2100]], y=info[[col_EI_2050, col_EI_2100]],
                marker={'color': [color, '#FFF'], 'size': 8, 'line': {'color': color, 'width': 2}},
                line={'color': color, 'width': 1, 'dash': 'solid'},
                mode='markers+lines', name=stripped_model,
                showlegend=False
            )





    ##############
    # 2b: CoEI
    ##############
    
    add_model_comparison(
        fig_CoEI, 2, f'CoEI {year}',
        narrative_left='More via demand red.', narrative_right='More via decarbon.',
        labelshift=1.2, **kwargs
    )

    fig_CoEI.add_scatter(
        x=[0,0.4490], y=[0,0.4490], showlegend=False,
        mode='lines', line={'dash': 'dot', 'color': '#DDD'}
    ).add_annotation(
        x=0.35, y=0.35, showarrow=False, textangle=-75,
        text='Demand red. dominant', xshift=-12, font_color='#888'
    ).add_annotation(
        x=0.35, y=0.35, showarrow=False, textangle=-75,
        text='Decarb. dominant', xshift=10, font_color='#888'
    )


    # Update layout
    (
        fig_CoEI
        .update_xaxes(
            col=1,
            gridcolor=gridcolor,
            title='Carbon intensity (red. from baseline)',
            title_standoff=40
        )
        .update_yaxes(
            col=1,
            gridcolor=gridcolor,
            title='Energy intensity (red. from baseline)',
            range=[-0.0113, 0.4490]
        )
        .update_xaxes(
            col=2,
            tickvals=np.arange(0.5, 0.901, 0.1),
            range=xrange,
            title='CI over (EI+CI) (red. from baseline)',
            title_standoff=40
        )
        .update_layout(
            width=860,
            height=400,
            margin={'l': 60, 'r': 30, 't': 50, 'b': 70},
            hovermode='closest'
        )
    )
    
    return fig_CoEI

## Transformation: % fossil fuel reduction

$$\text{FFR}(t)=\frac{\text{Prim.energy}_\text{fossil}(2020) - \text{Prim.energy}_\text{fossil, pol}(t)}{\text{Prim.energy}_\text{fossil}(2020)}$$

In [ ]:
def calc_fossil_fuel_reduction(data, year, pol, base='diag-base', var='Primary Energy|Fossil'):
    
    prim_energy_fossil_2020 = get(data, base, var, '2020')
    prim_energy_fossil_pol  = get(data, pol, var, year)
    
    return (prim_energy_fossil_2020 - prim_energy_fossil_pol) / prim_energy_fossil_2020

In [ ]:
def create_fig_FFR(year: str, xrange=None, exclude_models=None, models=models):
    
    if exclude_models is not None:
        models = models[~models['Full model'].str.contains(exclude_models)].copy()
        models['i'] = np.arange(len(models))

    fig_FFR = make_subplots(1, 2, horizontal_spacing=0.0, column_widths=[0.45, 0.55], subplot_titles=(
        f'<b>a.</b> Primary Energy decomposition ({year}) <br> ', f'<b>b.</b> FFR per model ({year})<br> '
    ), shared_yaxes=True)

    policy_scenario = 'diag-c80-gr5'

    ##############
    # 3a:
    ##############

    variables = ['Fossil|w/o CCS', 'Fossil|w/ CCS', 'Nuclear', 'Biomass|w/o CCS', 'Biomass|w/ CCS', 'Non-Biomass Renewables']
    labels = ['Fossil<br>w/o CCS', 'Fossil<br>w. CCS', 'Nuclear', 'Biomass<br>w/o CCS', 'Biomass<br>w. CCS', 'Renewables']
    for var_suffix in variables:
        var = f'Primary Energy|{var_suffix}'
        meta[f'{var} {year}'] = get(data, policy_scenario, var, year)

    # Replace w/o CCS variables by full variable if w/ CCS data is missing:
    for fuel in ['Fossil', 'Biomass']:
        var = f'Primary Energy|{fuel}'
        meta[f'{var} {year}'] = get(data, policy_scenario, var, year)
        missing = meta[f'{var}|w/ CCS {year}'].isna()
        meta.loc[missing, f'{var}|w/o CCS {year}'] = meta.loc[missing, f'{var} {year}']


    energy_colors = ['#d62728', '#d67a7a', '#ff7f0e', '#2ca02c', '#96d096', '#1f77b4']

    xpos = -0.05
    for var_suffix, label, color, labelwidth in zip(variables, labels, energy_colors, [0.09, 0.09, 0.1, 0.11, 0.11, 0.1]):
        col = f'Primary Energy|{var_suffix} {year}'
        energy_values = meta[col]

        var_values = models.merge(
            energy_values.reset_index(),
            how='left', left_on='Full model', right_on='Model'
        )
        fig_FFR.add_bar(
            y=var_values['i'], x=var_values[col],
            orientation='h', name=var, marker_color=color,
            showlegend=False
        )

        # Add legend
        height, width = 0.04, 0.04 * 400 / 840
        ypos=-0.1
        fig_FFR.add_shape(
            type='rect',
            xref='paper', yref='paper',
            x0=xpos-width/2, x1=xpos+width/2, y0=ypos-height/2, y1=ypos+height/2,
            fillcolor=color, line_width=0
        )
        fig_FFR.add_annotation(
            xref='paper', yref='paper', x=xpos+0.6*width, y=ypos,
            yanchor='top', yshift=12, xanchor='left', align='left',
            showarrow=False, text=label
        )
        xpos += labelwidth

    # check_values = models.merge(
    #     get(data, policy_scenario, 'Primary Energy', year).reset_index(),
    #     how='left', left_on='Full model', right_on='Model'
    # )
    # fig_FFR.add_scatter(
    #     x=check_values[year], y=check_values['i'],
    #     mode='markers', marker={'symbol': 'x', 'color': '#BBB'},
    #     showlegend=False
    # )


    ##############
    # 3b: FFR
    ##############

    col_FFR = f'FFR {year}'
    meta[col_FFR] = calc_fossil_fuel_reduction(data, year, policy_scenario)

    add_model_comparison(
        fig_FFR, 2, col_FFR,
        narrative_left='Small fossil fuel red.', narrative_right='Large fossil fuel red.',
        shared_yaxes=True, showlegendlines=False,
        exclude_models=exclude_models
    )



    # Update layout
    (
        fig_FFR
        .update_xaxes(
            col=1,
            title=f'Primary Energy (EJ in {year})',
            title_standoff=40
        )
        .update_xaxes(
            col=2,
            tickvals=np.arange(-0.2, 2, 0.2),
            title='FFR',
            range=xrange,
            title_standoff=40
        )
        .update_layout(
            barmode='stack',
            width=860,
            height=400,
            margin={'l': 60, 'r': 30, 't': 50, 'b': 70},
            hovermode='closest'
        )
    )
    return fig_FFR

## Inertia timescale

In [ ]:
# Calculate inertia timescale 

def linearInterp(row):
    years = row.index.astype(float)
    return np.trapz(row, x=years)


In [ ]:
def create_fig_IT(suffix: 'c80|c30'):

    fig_IT = make_subplots(1, 2, horizontal_spacing=0.02, column_widths=[0.4, 0.6], subplot_titles=(
        '<b>a.</b> Emissions gap vs excess emisssions<br> ', '<b>b.</b> Inertia Timescale per model<br> '
    ))

    exclude_models_IT = 'GEM-E3|DNE21|PROMETHEUS|TIAM_Grantham|COFFEE'


    ##############
    # 4a: 
    ##############

    cols_x = [f'Excess emissions {suffix}']
    cols_y = [f'Gap emissions {suffix}']
    for is_newest, selection in meta[
        ~meta[cols_x+cols_y].isna().any(axis=1) 
        & ~meta['Stripped model'].str.contains(exclude_models_IT)
    ].groupby('Newest'):
        for model, info in selection.iterrows():
            stripped_model = info['Stripped model']

            if stripped_model not in models.index:
                # Ignore entries that are not in `models`
                continue

            color = models.loc[stripped_model, 'Color'] if is_newest else '#BBB'

            fig_IT.add_scatter(
                x=info[cols_x], y=info[cols_y],
                marker={'color': [color, '#FFF'], 'size': 8, 'line': {'color': color, 'width': 2}},
                mode='markers', name=stripped_model,
                showlegend=False
            )



    ##############
    # 4b: IT
    ##############


    add_model_comparison(
        fig_IT, 2, f'IT {suffix}',
        narrative_left='Less inertia', narrative_right='More inertia',
        exclude_models=exclude_models_IT
    )



    # Update layout
    (
        fig_IT
        .update_xaxes(
            col=1,
            title='Cumulative excess emissions (GtCO<sub>2</sub>)'
        )
        .update_yaxes(
            col=1,
            title='Emissions difference (GtCO<sub>2</sub>/yr)'
        )
        .update_xaxes(
            col=2,
            tickvals=np.arange(0, 25.01, 5),
            title='Inertia Timescale (years)',
            title_standoff=40
        )
        .update_layout(
            width=860,
            height=400,
            margin={'l': 60, 'r': 30, 't': 50, 'b': 70},
            hovermode='closest'
        )
    )
    return fig_IT

## Cost per abatement value

In [ ]:
def set_value_from_var_column(data, meta, metacol, col, year, scenario):
    meta[col] = np.nan
    for model, info in meta.iterrows():
        var = info[metacol]
        selection = data[
            (data['Variable'] == var)
            & (data['Model'] == model)
            & (data['Scenario'] == scenario)
        ]
        if len(selection) > 0:
            meta.loc[model, col] = selection.iloc[0][year]

In [ ]:
def create_fig_CAV(year: str, xrange=None, **kwargs):
    fig_CAV = make_subplots(1, 2, horizontal_spacing=0.02, column_widths=[0.4, 0.6], subplot_titles=(
        '<b>a.</b> Costs vs abatement<br> ', f'<b>b.</b> CAV per model ({year})<br> '
    ))

    ##############
    # 5a: 
    ##############

    cols_x = ['Policy cost 2050 per GDP', 'Policy cost 2100 per GDP']
    cols_y = ['RAI c80 2050 CO2 FFI', 'RAI c80 2100 CO2 FFI']

    # Add background shape for 2050 and 2100
    for col_x, col_y, col_year in [(cols_x[0], cols_y[0], 2050), (cols_x[1], cols_y[1], 2100)]:
        selection = meta[~meta[[col_x, col_y]].isna().any(axis=1)]
        x_ellipse, y_ellipse = confidence_ellipse(selection[col_x], selection[col_y], 2)
        fig_CAV.add_scatter(
            x=x_ellipse, y=y_ellipse, fill='toself',
            showlegend=False, fillcolor='rgba(0,0,0,.08)', line_width=0
        )
        fig_CAV.add_annotation(
            text=col_year,
            x=max(x_ellipse), y=max(y_ellipse) if col_year == 2100 else min(y_ellipse),
            font_color='#999', showarrow=False
        )


    for is_newest, selection in meta[~meta[cols_x+cols_y].isna().any(axis=1)].groupby('Newest'):
        for model, info in selection.iterrows():
            stripped_model = info['Stripped model']

            if stripped_model not in models.index:
                # Ignore entries that are not in `models`
                continue

            color = models.loc[stripped_model, 'Color'] if is_newest else '#BBB'

            fig_CAV.add_scatter(
                x=info[cols_x], y=info[cols_y],
                marker={'color': [color, '#FFF'], 'size': 8, 'line': {'color': color, 'width': 2}},
                line={'color': color, 'width': 1, 'dash': 'solid'},
                mode='markers+lines', name=stripped_model,
                showlegend=False
            )





    ##############
    # 5b: CAV
    ##############

    add_model_comparison(
        fig_CAV, 2, f'CAV {year}',
        narrative_left='Less expensive', narrative_right='More expensive',
        **kwargs
    )



    # Update layout
    (
        fig_CAV
        .update_xaxes(
            col=1,
            gridcolor=gridcolor,
            title='Policy cost (% of GDP)',
            rangemode='tozero',
            tickformat='%'
        )
        .update_yaxes(
            col=1,
            gridcolor=gridcolor,
            title='Rel. abatement index',
            title_standoff=40,
            rangemode='tozero'
        )
        .update_xaxes(
            col=2,
            tickvals=np.arange(0, 1.81, 0.4),
            title='CAV',
            range=xrange,
            # title='CI over EI (rel. to baseline)',
            title_standoff=40
        )
        .update_layout(
            width=860,
            height=400,
            margin={'l': 60, 'r': 30, 't': 50, 'b': 70},
            hovermode='closest'
        )
    )
    return fig_CAV

ValueError: cannot reindex from a duplicate axis